# ❄️ End-to-end ML Demo ❄️

In this worfklow we will work through the following elements of a typical tabular machine learning pipeline.

### 1. Use Feature Store to track engineered features
* Store feature defintions in feature store for reproducible computation of ML features
      
### 2. Train two Models using the Snowflake ML APIs
* Baseline XGboost
* XGboost with optimal hyper-parameters identified via Snowflake ML distributed HPO methods

### 3. Register both models in Snowflake model registry
* Explore model registry capabilities such as **metadata tracking, inference, and explainability**
* Compare model metrics on train/test set to identify any issues of model performance or overfitting
* Tag the best performing model version as 'default' version
### 4. Set up Model Monitor to track 1 year of predicted and actual loan repayments
* **Compute performance metrics** such a F1, Precision, Recall
* **Inspect model drift** (i.e. how much has the average predicted repayment rate changed day-to-day)
* **Compare models** side-by-side to understand which model should be used in production
* Identify and understand **data issues**

### 5. Track data and model lineage throughout
* View and understand
  * The **origin of the data** used for computed features
  * The **data used** for model training
  * The **available model versions** being monitored

In [ ]:
#!pip install shap snowflake-ml-python==1.11.0


### Import Libraries for End-to-End ML Workflow

Import all required libraries for data processing, model training, MLOps, and visualization in Snowflake.

**Categories:**

**Standard ML Libraries:**
- `pandas`, `numpy`: Data manipulation
- `sklearn`, `xgboost`: Model training and metrics
- `shap`: Model explainability (Shapley values)
- `streamlit`: Interactive visualizations

**Snowflake ML (MLOps):**
- `Registry`: Model versioning and deployment
- `tune`, `get_tuner_context`: Distributed hyperparameter optimization
- `FeatureStore`, `FeatureView`, `Entity`: Feature engineering and management

**Snowpark (Distributed Processing):**
- `DataFrame`: Lazy DataFrames executing in Snowflake warehouse
- `functions`: Feature engineering (month, dayofweek, avg, sql_expr, etc.)
- `Window`: Aggregations across row groups (county averages, rolling windows)
- `get_active_session`: Snowflake connection and context

**Together:** Enable complete ML workflow from data to production without moving data outside Snowflake.


In [1]:
import pandas as pd
import numpy as np
import sklearn
import math
import pickle
import shap
from datetime import datetime
import streamlit as st
from xgboost import XGBClassifier

# Snowpark ML
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.tune import get_tuner_context
from snowflake.ml.modeling import tune
from entities import search_algorithm

#Snowflake feature store
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

# Snowpark session
from snowflake.snowpark import DataFrame
from snowflake.snowpark.functions import col, to_timestamp, min, max, month, dayofweek, dayofyear, avg, date_add, sql_expr
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark import Window

# warning suppresion
import warnings; warnings.simplefilter('ignore')

#setup snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()
session



In [ ]:
#Update this VERSION_NUM to version your features, models etc!
VERSION_NUM = '0'
DB = "SANDBOX" 
SCHEMA = "MEDPACE_HOL" 
COMPUTE_WAREHOUSE = "APP_WH" 

In [ ]:
-- Using Warehouse, Database, and Schema created during Setup
USE WAREHOUSE APP_WH;

USE DATABASE SANDBOX;

USE SCHEMA MEDPACE_HOL;

In [ ]:
CREATE STAGE IF NOT EXISTS MEDPACE_HOL.MORTGAGE_LENDING_DEMO_STAGE 
	DIRECTORY = ( ENABLE = true );

### Create a file format to ingest CSVs while preserving headers.

In [ ]:
-- create csv format
CREATE FILE FORMAT IF NOT EXISTS SANDBOX.MEDPACE_HOL.CSV_W_HEADERS 
   PARSE_HEADER = true
    TYPE = 'CSV';

In [ ]:
    CREATE OR REPLACE TABLE MORTGAGE_LENDING_DEMO_DATA
        USING TEMPLATE (
            SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
            FROM TABLE(
                INFER_SCHEMA(
                    LOCATION => '@sandbox.medpace_hol.mortgage_lending_demo_stage/MORTGAGE_LENDING_DEMO_DATA.csv',
                    FILE_FORMAT => 'CSV_W_HEADERS'
                )
            )
        );

In [ ]:
desc table MORTGAGE_LENDING_DEMO_DATA;

In [ ]:
COPY INTO MORTGAGE_LENDING_DEMO_DATA
FROM @sandbox.medpace_hol.mortgage_lending_demo_stage/MORTGAGE_LENDING_DEMO_DATA.csv
FILE_FORMAT = (FORMAT_NAME = 'CSV_W_HEADERS')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE

## Observe Snowflake Snowpark table properties

In [ ]:
df = session.table("MORTGAGE_LENDING_DEMO_DATA")
df.select(min('TS'), max('TS'))


### Calculate Timestamp Adjustment

Shift historical timestamps to make the dataset current for realistic model monitoring demos.

**Process:**

1. **Get current date:** `datetime.now()`
2. **Find dataset max date:** Extract latest timestamp from `TS` column
3. **Calculate difference:** `timedelta = current_time - df_max_time`
4. **Preview adjusted range:** Show min/max after adding `timedelta.days - 1` to all timestamps

**Example:**


In [ ]:
#Get current date and time
current_time = datetime.now()
df_max_time = datetime.strptime(str(df.select(max("TS")).collect()[0][0]), "%Y-%m-%d %H:%M:%S.%f")

#Find delta between latest existing timestamp and today's date
timedelta = current_time- df_max_time

#Update timestamps to represent last ~1 year from today's date
df.select(min(date_add(to_timestamp("TS"), timedelta.days-1)), max(date_add(to_timestamp("TS"), timedelta.days-1)))

## Feature Engineering with Snowpark APIs


### Feature Engineering with Snowpark

Create 10 ML features using a dictionary-based approach for clean, maintainable code.

**Feature Categories:**

**1. Temporal (4):** Adjusted timestamp + seasonality (MONTH, DAY_OF_YEAR, DOTW)
- Capture seasonal patterns and weekly trends

**2. Financial (3):** Convert units + affordability ratio
- `LOAN_AMOUNT`, `INCOME` (thousands → dollars)
- `INCOME_LOAN_RATIO` = affordability metric (higher = lower risk)

**3. Geographic Context (2):** Window functions for county-level comparison
- `MEAN_COUNTY_INCOME`: Average per county
- `HIGH_INCOME_FLAG`: Binary (above/below county average)
- Same income means different things in different areas!

**4. Time-Series (1):** Rolling 30-day average using SQL window
- `AVG_THIRTY_DAY_LOAN_AMOUNT`: Market trend indicator
- Detects hot/cooling markets by county

**Key Techniques:**
- Window functions: `avg(...).over(Window.partition_by(...))`
- Rolling windows: `RANGE BETWEEN INTERVAL '30 DAYS' PRECEDING...`
- Type conversions: `(boolean).astype(IntegerType())`
- Derived ratios: Division of existing features

**Result:** All transformations execute in Snowflake warehouse (no data movement), creating 10 engineered features in one operation via `df.with_columns()`.


In [ ]:
#Create a dict with keys for feature names and values containing transform code

feature_eng_dict = dict()

#Timstamp features
feature_eng_dict["TIMESTAMP"] = date_add(to_timestamp("TS"), timedelta.days-1)
feature_eng_dict["MONTH"] = month("TIMESTAMP")
feature_eng_dict["DAY_OF_YEAR"] = dayofyear("TIMESTAMP") 
feature_eng_dict["DOTW"] = dayofweek("TIMESTAMP")

# df= df.with_columns(feature_eng_dict.keys(), feature_eng_dict.values())

#Income and loan features
feature_eng_dict["LOAN_AMOUNT"] = col("LOAN_AMOUNT_000s")*1000
feature_eng_dict["INCOME"] = col("APPLICANT_INCOME_000s")*1000
feature_eng_dict["INCOME_LOAN_RATIO"] = col("INCOME")/col("LOAN_AMOUNT")

county_window_spec = Window.partition_by("COUNTY_NAME")
feature_eng_dict["MEAN_COUNTY_INCOME"] = avg("INCOME").over(county_window_spec)
feature_eng_dict["HIGH_INCOME_FLAG"] = (col("INCOME")>col("MEAN_COUNTY_INCOME")).astype(IntegerType())

feature_eng_dict["AVG_THIRTY_DAY_LOAN_AMOUNT"] =  sql_expr("""AVG(LOAN_AMOUNT) OVER (PARTITION BY COUNTY_NAME ORDER BY TIMESTAMP  
                                                            RANGE BETWEEN INTERVAL '30 DAYS' PRECEDING AND CURRENT ROW)""")

df = df.with_columns(feature_eng_dict.keys(), feature_eng_dict.values())
df.show(3)

### View Snowpark Query Execution Plan

Display the SQL execution plan that Snowflake will use to compute the DataFrame with all engineered features.

---

#### **What `df.explain()` Does:**

Shows the **logical and physical query plan** that Snowpark generates from all DataFrame operations, including:
- Feature transformations
- Window functions
- Column additions
- Data sources

**Purpose:**
- 🔍 **Understand query structure**: See how Snowpark translates Python to SQL
- ⚡ **Optimize performance**: Identify expensive operations
- 📊 **Debug issues**: Understand execution flow
- 🎓 **Learn Snowpark**: See the generated SQL behind the scenes

---

#### **What You'll See:**

The output shows a **tree-like structure** of operations that will be executed in Snowflake:

In [ ]:
df.explain()

## Create a Snowflake Feature Store

### Snowflake Feature Store

Create or connect to a Feature Store for centralized ML feature management with automatic refresh and lineage tracking.

---

#### **What is a Feature Store?**

A centralized system for managing ML features throughout their lifecycle:
- **Store** feature definitions (transformation logic)
- **Refresh** features automatically from source data
- **Track** lineage from raw data → features → models
- **Share** features across multiple models and teams
- **Version** features for reproducibility

**Think of it as:** GitHub for ML features - centralized, versioned, and reusable.

---

#### **Key Architecture Insight:**

In Snowflake, a **Feature Store IS a schema**:

**Benefits of native integration:**
- ✅ No external system required
- ✅ Leverage existing Snowflake governance (RBAC, access control)
- ✅ Query features directly with SQL
- ✅ Automatic integration with Model Registry and ML Lineage

---

#### **Parameters Explained:**

**`session`**: Active Snowflake connection for executing operations

**`database` + `name`**: Location where Feature Store lives (schema within database)

**`default_warehouse`**: Compute cluster for feature operations
- Powers automatic feature refresh
- Executes feature transformations
- Serves features for training/inference

**`creation_mode=CREATE_IF_NOT_EXIST`**: Idempotent behavior
- **First run**: Creates new Feature Store schema
- **Subsequent runs**: Connects to existing Feature Store
- **Result**: Safe to run notebook multiple times without errors

---

#### **What Gets Created:**

Behind the scenes, Snowflake:
1. Creates (or connects to) a schema as the Feature Store
2. Initializes metadata structures for tracking entities and feature views
3. Links Feature Store to the specified warehouse for compute

**Ready for next steps:**
- Register Entities (e.g., LOAN_ID)
- Create Feature Views (with automatic refresh schedules)
- Generate datasets for model training

---

#### **Why Feature Store Matters:**

**Without Feature Store:**
- ❌ Feature definitions scattered across notebooks
- ❌ Manual feature refresh (stale data)
- ❌ Duplicate feature code across projects
- ❌ No lineage tracking (where did this feature come from?)
- ❌ Hard to reuse features across models

**With Feature Store:**
- ✅ Centralized feature repository
- ✅ Automatic refresh (features stay current)
- ✅ Reuse features across multiple models
- ✅ Complete lineage (data → features → models)
- ✅ Feature documentation and discovery
- ✅ Consistent feature computation (training = inference)

---

#### **Next Steps:**

After initializing the Feature Store, we'll:
1. **Define Entities** (e.g., LOAN_ID) - what features describe
2. **Create Feature Views** - collections of features with refresh schedules
3. **Generate Datasets** - combine features for model training
4. **Track Lineage** - automatic lineage to models in Model Registry

In [ ]:
fs = FeatureStore(
    session=session, 
    database=DB, 
    name=SCHEMA, 
    default_warehouse=COMPUTE_WAREHOUSE,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

In [ ]:
fs.list_entities()

### Register Entity in Feature Store

This cell creates and registers an **Entity** using idempotent logic to ensure safe re-execution.

---

#### **What is an Entity?**

An **Entity** represents a **business object** around which features are organized.

- **Entity**: `LOAN_ENTITY`
- **Represents**: Individual mortgage loans  
- **Identified by**: `LOAN_ID` (the join key)

**Think of it as:** "What are we making predictions about?"  
→ In this case: Each **LOAN** has features (income, loan amount, location, etc.)  
→ We want to predict loan approval for each **LOAN**  
→ Therefore: **LOAN** is our Entity

---

#### **🔑 Key Components**

**`name = "LOAN_ENTITY"`**
- Unique identifier for this entity in the Feature Store
- Used to reference this entity when creating Feature Views

**`join_keys = ["LOAN_ID"]`**
- Column(s) that uniquely identify each instance of this entity
- Used to join features together when generating training datasets

**`desc = "Features defined on a per loan level"`**
- Documentation/metadata describing what this entity represents

---

#### **⚙️ Why Try/Except Pattern?**

**Purpose: Idempotency** - Running this cell multiple times produces the same result without errors.

| Run | Behavior | Output |
|-----|----------|--------|
| 1st | Entity doesn't exist → Create & register | `"Registered new entity"` |
| 2nd+ | Entity exists → Retrieved successfully | `"Retrieved existing entity"` |

**Benefits:**
- ✅ No duplicates - Won't register the same entity twice
- ✅ No errors - Won't crash if entity already exists  
- ✅ Notebook re-runability - Safe to execute multiple times
- ✅ Development friendly - Can iterate without manual cleanup

---

#### **After Execution**

The Feature Store now contains:
Feature Store

└─> Entities

└─> LOAN_ENTITY

    ├─ Join Keys: [LOAN_ID]

└─ Description: "Features defined on a per loan level"

In [ ]:
#First try to retrieve an existing entity definition, if not define a new one and register
try:
    #retrieve existing entity
    loan_id_entity = fs.get_entity('LOAN_ENTITY') 
    print('Retrieved existing entity')
except:
#define new entity
    loan_id_entity = Entity(
        name = "LOAN_ENTITY",
        join_keys = ["LOAN_ID"],
        desc = "Features defined on a per loan level")
    #register
    fs.register_entity(loan_id_entity)
    print("Registered new entity")

### Prepare Feature DataFrame for Feature View

Create a clean DataFrame containing only the columns needed for the Feature View: entity join key, timestamp, and engineered features.

---

#### **What This Does:**

```python
feature_df = df.select(["LOAN_ID"] + list(feature_eng_dict.keys()))
```

**Selects:**
- **`LOAN_ID`**: Entity join key (required)
- **All engineered features** from `feature_eng_dict`:
  - `TIMESTAMP` (required for point-in-time joins)
  - `MONTH`, `DAY_OF_YEAR`, `DOTW` (temporal features)
  - `LOAN_AMOUNT`, `INCOME`, `INCOME_LOAN_RATIO` (financial features)
  - `MEAN_COUNTY_INCOME`, `HIGH_INCOME_FLAG` (geographic features)
  - `AVG_THIRTY_DAY_LOAN_AMOUNT` (time-series feature)

**Excludes:**
- ❌ Raw columns (e.g., `LOAN_AMOUNT_000s`, `APPLICANT_INCOME_000s`)
- ❌ Target variable (stored separately, not in Feature View)
- ❌ Unnecessary metadata

---

#### **Why Create This Subset?**

**Feature Store requires:**
1. ✅ Entity join key(s) - `LOAN_ID`
2. ✅ Timestamp column - `TIMESTAMP` (for point-in-time correctness)
3. ✅ Feature columns - All engineered features ready for ML

This DataFrame provides exactly those components for Feature View registration.

---

#### **Preview Output:**

`.show(5)` displays first 5 rows to verify:
- ✅ All expected columns are present
- ✅ Feature values look correct
- ✅ Data is ready for Feature View definition

In [ ]:
#Create a dataframe with just the ID, timestamp, and engineered features. We will use this to define our feature view
feature_df = df.select(["LOAN_ID"]+list(feature_eng_dict.keys()))
feature_df.show(5)

### Define and Register Feature View

Create a Feature View object that wraps the feature DataFrame with metadata, documentation, and configuration, then register it permanently in the Feature Store.

---

#### **Step 1: Define Feature View**

```python
loan_fv = FeatureView(
    name="Mortgage_Feature_View",
    entities=[loan_id_entity],
    feature_df=feature_df,
    timestamp_col="TIMESTAMP",
    refresh_freq="1 day"
)
```

**Key Parameters:**
- **`name`**: Unique identifier for this Feature View
- **`entities`**: Links to `LOAN_ENTITY` (defines join key: `LOAN_ID`)
- **`feature_df`**: Uses the DataFrame from previous cell as data source
- **`timestamp_col`**: Enables point-in-time correctness for training data generation
- **`refresh_freq`**: Automatically refreshes features daily

---

#### **Step 2: Add Feature Descriptions**

```python
loan_fv = loan_fv.attach_feature_desc({...})
```

**Purpose:**
- 📝 Documents each feature for team collaboration
- 🔍 Improves Feature Store discoverability
- 📊 Enhances governance and lineage tracking

---

#### **Step 3: Register in Feature Store**

```python
loan_fv = fs.register_feature_view(loan_fv, version=VERSION_NUM, overwrite=True)
```

**What happens:**
- ✅ Persists Feature View definition permanently
- ✅ Creates versioned backend objects in Snowflake
- ✅ Enables reuse across notebooks, users, and models
- ✅ Sets up automatic daily refresh schedule
- ✅ Makes features discoverable and accessible organization-wide

**`overwrite=True`**: Ensures idempotency - safe to re-run without errors

---

#### **Result:**

Features are now registered, versioned, documented, and ready for:
- 🎯 Training dataset generation
- 🔄 Automatic refresh and maintenance
- 🤝 Team-wide collaboration
- 📈 Model training and inference

In [ ]:
#define and register feature view
loan_fv = FeatureView(
    name="Mortgage_Feature_View",
    entities=[loan_id_entity],
    feature_df=feature_df,
    timestamp_col="TIMESTAMP",
    refresh_freq="1 day")

#add feature level descriptions

loan_fv = loan_fv.attach_feature_desc(
    {
        "MONTH": "Month of loan",
        "DAY_OF_YEAR": "Day of calendar year of loan",
        "DOTW": "Day of the week of loan",
        "LOAN_AMOUNT": "Loan amount in $USD",
        "INCOME": "Household income in $USD",
        "INCOME_LOAN_RATIO": "Ratio of LOAN_AMOUNT/INCOME",
        "MEAN_COUNTY_INCOME": "Average household income aggregated at county level",
        "HIGH_INCOME_FLAG": "Binary flag to indicate whether household income is higher than MEAN_COUNTY_INCOME",
        "AVG_THIRTY_DAY_LOAN_AMOUNT": "Rolling 30 day average of LOAN_AMOUNT"
    }
)

loan_fv = fs.register_feature_view(loan_fv, version=VERSION_NUM, overwrite=True)

In [ ]:
fs.list_feature_views()

### Generate Feature Store UI Link

Dynamically construct a clickable URL to inspect the newly registered Feature View in Snowflake's Feature Store UI (Snowsight).

---

#### **What This Does:**

```python
org_name = session.sql('SELECT CURRENT_ORGANIZATION_NAME()').collect()[0][0]
account_name = session.sql('SELECT CURRENT_ACCOUNT_NAME()').collect()[0][0]
```

**Retrieves current Snowflake context:**
- **`org_name`**: Your Snowflake organization name
- **`account_name`**: Your Snowflake account name

**Constructs URL:**
```python
st.write(f'https://app.snowflake.com/{org_name}/{account_name}/#/features/database/{DB}/store/{SCHEMA}')
```

Builds a direct link to the Feature Store UI for this specific database and schema.

---

#### **Why This is Useful:**

**Click the link to explore:**
- 🔍 **Feature View details** (name, version, entity, refresh schedule)
- 📊 **Feature catalog** (all features with descriptions)
- 📈 **Lineage tracking** (data sources and dependencies)
- 🔄 **Refresh status** (last refresh, next scheduled refresh)
- 🤝 **Team collaboration** (discoverable by other users)

---

#### **Why Dynamic Construction?**

✅ **Portable** - Works across different Snowflake accounts  
✅ **Environment-agnostic** - Adapts to dev, test, prod  
✅ **No hardcoding** - Automatically uses current context  
✅ **Shareable** - Notebook works for all team members

---

**Click the generated link to visually inspect your Feature View in Snowsight!** 🚀

In [ ]:
#Create link to feature store UI to inspect newly created feature view!
org_name = session.sql('SELECT CURRENT_ORGANIZATION_NAME()').collect()[0][0]
account_name = session.sql('SELECT CURRENT_ACCOUNT_NAME()').collect()[0][0]

st.write(f'https://app.snowflake.com/{org_name}/{account_name}/#/features/database/{DB}/store/{SCHEMA}')

### Generate Dataset from Feature Store

Create a complete dataset by joining the spine DataFrame with registered Feature Views using point-in-time correctness.

---

#### **What This Does:**

```python
ds = fs.generate_dataset(
    name=f"MORTGAGE_DATASET_EXTENDED_FEATURES_{VERSION_NUM}",
    spine_df=df.select("LOAN_ID", "TIMESTAMP", "LOAN_PURPOSE_NAME", "MORTGAGERESPONSE"),
    features=[loan_fv],
    spine_timestamp_col="TIMESTAMP",
    spine_label_cols=["MORTGAGERESPONSE"]
)
```

---

#### **Key Parameters:**

**`name`**: Unique identifier for this dataset (versioned with `VERSION_NUM`)

**`spine_df`**: The "backbone" DataFrame containing:
- **`LOAN_ID`**: Join key to match with Feature View
- **`TIMESTAMP`**: For point-in-time feature lookups
- **`LOAN_PURPOSE_NAME`**: Additional contextual feature
- **`MORTGAGERESPONSE`**: Target variable (label)

**`features`**: List of Feature Views to join (in this case: `[loan_fv]`)

**`spine_timestamp_col`**: Column for point-in-time correctness

**`spine_label_cols`**: Target variable(s) to include in final dataset

---

#### **🎯 What Happens Under the Hood:**

1. **Point-in-Time Join**: For each row in `spine_df`, fetch features from `loan_fv` as they existed at that `TIMESTAMP`
2. **Prevents Data Leakage**: Ensures features reflect only information available at that point in time
3. **Combines Data**: Joins spine columns + Feature View features + labels
4. **Registers Dataset**: Saves as a reusable, versioned dataset in Feature Store

---

#### **📊 Result:**

A complete ML-ready dataset containing:
- ✅ Entity keys (`LOAN_ID`)
- ✅ Timestamps (`TIMESTAMP`)
- ✅ Additional spine features (`LOAN_PURPOSE_NAME`)
- ✅ All engineered features from Feature View (MONTH, INCOME, INCOME_LOAN_RATIO, etc.)
- ✅ Target variable (`MORTGAGERESPONSE`)

**Can be used for:** Model training, testing, validation, inference, or exploratory analysis

---

#### **💡 Why Use Feature Store for Dataset Generation?**

✅ **Point-in-time correctness** - No data leakage  
✅ **Feature reuse** - Consistent features across use cases  
✅ **Version control** - Reproducible datasets  
✅ **Lineage tracking** - Full visibility into data sources  
✅ **Consistency** - Same feature definitions everywhere

In [ ]:
ds = fs.generate_dataset(
    name=f"MORTGAGE_DATASET_EXTENDED_FEATURES_{VERSION_NUM}",
    spine_df=df.select("LOAN_ID", "TIMESTAMP", "LOAN_PURPOSE_NAME","MORTGAGERESPONSE"), #only need the features used to fetch rest of feature view
    features=[loan_fv],
    spine_timestamp_col="TIMESTAMP",
    spine_label_cols=["MORTGAGERESPONSE"]
)

### Load Dataset as Snowpark DataFrame

Materialize the registered dataset from Feature Store into a Snowpark DataFrame for analysis and modeling.

---

#### **What This Does:**

```python
ds_sp = ds.read.to_snowpark_dataframe()
```

**Converts the dataset object to a Snowpark DataFrame:**
- **`ds`**: Feature Store Dataset object (metadata/reference)
- **`ds.read.to_snowpark_dataframe()`**: Materializes the actual data
- **`ds_sp`**: Snowpark DataFrame ready for manipulation

**Key characteristics:**
- ✅ Data stays in Snowflake (no movement to client)
- ✅ Lazy evaluation (efficient query execution)
- ✅ Can handle large datasets
- ✅ Ready for transformations, splits, and ML pipelines

---

#### **Preview Data:**

```python
ds_sp.show(5)
```

Displays first 5 rows to verify the dataset contains:
- ✅ Spine columns (LOAN_ID, TIMESTAMP, LOAN_PURPOSE_NAME)
- ✅ All engineered features from Feature View
- ✅ Target variable (MORTGAGERESPONSE)

---

#### **Dataset Object vs Snowpark DataFrame:**

| `ds` (Dataset Object) | `ds_sp` (Snowpark DataFrame) |
|-----------------------|------------------------------|
| Feature Store metadata | Materialized data |
| Registered definition | Ready for manipulation |
| Reference/pointer | Actual rows and columns |
| Can't transform directly | Can filter, join, transform |

---

**Next:** Use `ds_sp` for EDA, train/test split, preprocessing, and model training.

In [ ]:
ds_sp = ds.read.to_snowpark_dataframe()
ds_sp.show(5)

### Preprocess Categorical Features

Apply one-hot encoding to convert categorical (string) columns into numerical format required by ML algorithms.

---

#### **What This Does:**

**1. Identify Categorical Columns**
```python
OHE_COLS = ds_sp.select([col.name for col in ds_sp.schema if col.datatype == StringType()]).columns
```
Automatically detects all string/categorical columns (e.g., `LOAN_PURPOSE_NAME`, `COUNTY_NAME`)

**2. Apply One-Hot Encoding**
```python
snowml_ohe = snowml.OneHotEncoder(input_cols=OHE_COLS, output_cols=OHE_COLS, drop_input_cols=True)
ds_sp_ohe = snowml_ohe.fit(ds_sp).transform(ds_sp)
```
- **`fit()`**: Learns all unique categories in each column
- **`transform()`**: Creates binary (0/1) columns for each category
- **`drop_input_cols=True`**: Removes original string columns

**3. Clean Column Names**
```python
rename_dict = {col: col.replace('"','').replace(' ', '_') for col in ds_sp_ohe.columns if '"' in col}
ds_sp_ohe = ds_sp_ohe.rename(rename_dict)
```
Removes quotes and replaces spaces with underscores for SQL compatibility

---

#### **Example Transformation:**

**Before:** `LOAN_PURPOSE_NAME = "Purchase"`  
**After:** `LOAN_PURPOSE_NAME_Purchase = 1`, `LOAN_PURPOSE_NAME_Refinance = 0`, `LOAN_PURPOSE_NAME_Home_Improvement = 0`

---

**Result:** `ds_sp_ohe` contains only numerical features, ready for train/test split and model training.

In [ ]:
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.snowpark.types import StringType

OHE_COLS = ds_sp.select([col.name for col in ds_sp.schema if col.datatype ==StringType()]).columns
OHE_POST_COLS = [i+"_OHE" for i in OHE_COLS]


# Encode categoricals to numeric columns
snowml_ohe = snowml.OneHotEncoder(input_cols=OHE_COLS, output_cols = OHE_COLS, drop_input_cols=True)
ds_sp_ohe = snowml_ohe.fit(ds_sp).transform(ds_sp)

#Rename columns to avoid double nested quotes and white space chars
rename_dict = {}
for i in ds_sp_ohe.columns:
    if '"' in i:
        rename_dict[i] = i.replace('"','').replace(' ', '_')

ds_sp_ohe = ds_sp_ohe.rename(rename_dict)
ds_sp_ohe.columns

### Split Data into Training and Test Sets

Divide the dataset into training (70%) and test (30%) sets for model development and evaluation.

---

#### **What This Does:**

```python
train, test = ds_sp_ohe.random_split(weights=[0.70, 0.30], seed=0)
```

**Parameters:**
- **`weights=[0.70, 0.30]`**: 70% of data for training, 30% for testing
- **`seed=0`**: Fixed random seed for reproducibility

---

#### **Why Split the Data?**

- **Training set (70%)**: Used to train the model and learn patterns
- **Test set (30%)**: Used to evaluate model performance on unseen data

**Purpose:** Ensures the model can generalize to new data and prevents overfitting.

---

**Result:** Two separate Snowpark DataFrames ready for model training and evaluation.

In [ ]:
train, test = ds_sp_ohe.random_split(weights=[0.70, 0.30], seed=0)

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
from snowflake.snowpark.functions import col, count, when

# Summary of TIMESTAMP column health
timestamp_check = train.select(
    count("*").alias("total_rows"),
    count("TIMESTAMP").alias("non_null_timestamps"),
    count(when(col("TIMESTAMP").is_null(), 1)).alias("null_timestamps")
).collect()[0]

print(f"Total rows: {timestamp_check['TOTAL_ROWS']}")
print(f"Non-null timestamps: {timestamp_check['NON_NULL_TIMESTAMPS']}")
print(f"Null timestamps: {timestamp_check['NULL_TIMESTAMPS']}")
print(f"Timestamp coverage: {timestamp_check['NON_NULL_TIMESTAMPS'] / timestamp_check['TOTAL_ROWS'] * 100:.2f}%")

### Convert to Pandas DataFrames

Convert Snowpark DataFrames to pandas DataFrames for local processing.

---

```python
train_pd = train.to_pandas()
test_pd = test.to_pandas()
```

**What this does:**
- Downloads data from Snowflake to local memory as pandas DataFrames
- Required for certain ML operations that need in-memory data structures
- Enables use of pandas-based libraries and methods

**Note:** Data is moved from Snowflake warehouse to the notebook's local environment. For large datasets, consider keeping data in Snowpark format when possible to leverage Snowflake's compute resources.

In [ ]:
train_pd = train.to_pandas()
test_pd = test.to_pandas()

## Model Training

### Train Baseline XGBoost Classifier

Define and train an **intentionally overfit** baseline XGBoost model to demonstrate the impact of poor hyperparameter tuning.

---

#### **Baseline Model Configuration:**

```python
xgb_base = XGBClassifier(
    max_depth=50,
    n_estimators=3,
    learning_rate=0.75,
    booster='gbtree'
)
```

**Hyperparameters (Intentionally Suboptimal):**
- **`max_depth=50`**: Extremely deep trees (typical: 3-10) → **overfitting**
- **`n_estimators=3`**: Very few boosting rounds (typical: 50-500) → **underfitting individual trees**
- **`learning_rate=0.75`**: Aggressive learning rate (typical: 0.01-0.3) → **overshooting optimal weights**
- **`booster='gbtree'`**: Tree-based gradient boosting algorithm

---

#### **⚠️ Why These Parameters Are Poor:**

| Parameter | Baseline Value | Typical Range | Problem |
|-----------|----------------|---------------|---------|
| `max_depth` | 50 | 3-10 | Trees memorize training data |
| `n_estimators` | 3 | 50-500 | Too few trees to learn patterns |
| `learning_rate` | 0.75 | 0.01-0.3 | Too aggressive, overshoots |

**Expected Result:** High training accuracy, poor test accuracy (classic overfitting)

---

#### **🎯 Purpose of This Baseline:**

1. **Establish comparison point** - Measure performance before optimization
2. **Demonstrate overfitting** - Show train/test performance gap
3. **Motivate HPO** - Create a clear need for hyperparameter optimization
4. **Educational value** - Teach the importance of proper tuning


**Next Steps:** Train this baseline model, evaluate on train/test sets, then compare against an optimized model trained with Distributed Hyperparameter Optimization (HPO).

In [ ]:
#Define model config
xgb_base = XGBClassifier(
    max_depth=50,
    n_estimators=3,
    learning_rate = 0.75,
    booster = 'gbtree')

### Prepare Training Data and Fit Baseline Model

Split the training data into features (X) and target (y), then train the baseline XGBoost classifier.

---

#### **Prepare Features and Target:**

```python
X_train_pd = train_pd.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"], axis=1)
y_train_pd = train_pd.MORTGAGERESPONSE
```

**Features (`X_train_pd`)**: All engineered and one-hot encoded columns, excluding:
- `TIMESTAMP` - Not used for prediction (temporal identifier only)
- `LOAN_ID` - Entity identifier, not a predictive feature
- `MORTGAGERESPONSE` - Target variable

**Target (`y_train_pd`)**: `MORTGAGERESPONSE` - Binary label indicating loan approval (1) or denial (0)

---

#### **Train the Model:**

```python
xgb_base.fit(X_train_pd, y_train_pd)
```

Trains the XGBoost classifier on the training data using the suboptimal hyperparameters defined previously.

---

**Result:** A trained baseline model that will likely overfit the training data (to be evaluated in the next cell).

In [ ]:
#Split train data into X, y
X_train_pd = train_pd.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"],axis=1) #remove
y_train_pd = train_pd.MORTGAGERESPONSE

#train model
xgb_base.fit(X_train_pd,y_train_pd)

### ✅ Model Training Complete

The output displays the trained XGBoost model object with its configuration parameters.

#### **Confirmed Hyperparameters:**

**Custom Settings (Intentionally Suboptimal):**
- `max_depth=50` - Extremely deep trees (overfitting risk)
- `n_estimators=3` - Very few boosting rounds
- `learning_rate=0.75` - Aggressive learning rate
- `booster='gbtree'` - Tree-based gradient boosting

**Default Settings:**
- Most other parameters set to `None` (using XGBoost defaults)
- `enable_categorical=False` - All features treated as numerical
- `random_state=None` - No fixed random seed

---

#### **What This Means:**

✅ **Training successful** - Model is fitted and ready for predictions  
✅ **Configuration active** - Your specified hyperparameters are applied  
✅ **Using defaults** - Unspecified parameters use XGBoost's standard values

---

**Next:** Evaluate this baseline model's performance on training and test data to observe overfitting behavior.

### Evaluate Baseline Model Performance on Training Data

Generate predictions on the training set and calculate classification metrics to assess baseline model performance.

#### **Generate Predictions:**

```python
train_preds_base = xgb_base.predict(X_train_pd)
```

Uses the trained baseline model to predict loan approval outcomes on the training data.


#### **Calculate Performance Metrics:**

```python
f1_base_train = round(f1_score(y_train_pd, train_preds_base), 4)
precision_base_train = round(precision_score(y_train_pd, train_preds_base), 4)
recall_base_train = round(recall_score(y_train_pd, train_preds_base), 4)
```

**Metrics Explained:**
- **F1 Score**: Harmonic mean of precision and recall (balanced measure)
- **Precision**: Of all predicted approvals, how many were actually approved? (PPV)
- **Recall**: Of all actual approvals, how many did we predict? (Sensitivity)


#### **Expected Result:**

**High training performance** (likely F1 > 0.95) - The overfit model should perform very well on data it was trained on.

**Next step:** Evaluate on test data to reveal the performance drop-off and confirm overfitting.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
train_preds_base = xgb_base.predict(X_train_pd) # Generate predictions on training set

f1_base_train = round(f1_score(y_train_pd, train_preds_base),4)
precision_base_train = round(precision_score(y_train_pd, train_preds_base),4)
recall_base_train = round(recall_score(y_train_pd, train_preds_base),4)

print(f'F1: {f1_base_train} \nPrecision {precision_base_train} \nRecall: {recall_base_train}')

## Model Registry

Snowflake's Model Registry provides centralized model management with metadata tracking, lifecycle management, versioning, and deployment capabilities.

#### **Key Capabilities:**

- 📝 **Log models** with metadata, metrics, and tags
- 🔄 **Manage lifecycles** - version, promote, retire models
- 🚀 **Serve models** from Snowflake Warehouses or Snowpark Container Services
- 📊 **Enable monitoring** - track model performance and drift over time
- 🔗 **Maintain lineage** - track data sources and dependencies

#### **Initialize Model Registry:**

```python
from snowflake.ml.registry import Registry

model_name = f"MORTGAGE_LENDING_MLOPS_{VERSION_NUM}"

model_registry = Registry(
    session=session, 
    database_name=DB, 
    schema_name=SCHEMA,
    options={"enable_monitoring": True}
)
```

**Configuration:**
- **`model_name`**: Unique identifier for this model family (versioned with `VERSION_NUM`)
- **`database_name`**: Database where models will be stored (`SANDBOX`)
- **`schema_name`**: Schema for model objects (`MEDPACE_HOL`)
- **`enable_monitoring=True`**: Enables drift detection and performance monitoring

#### **What This Creates:**

A Model Registry instance connected to your Snowflake account, ready to:
- ✅ Log baseline and optimized model versions
- ✅ Track metrics (F1, precision, recall)
- ✅ Set tags (PROD, DEV, etc.)
- ✅ Manage default versions
- ✅ Monitor model performance over time

**Next:** Log the baseline model with training metrics and metadata.

In [ ]:
#Create a snowflake model registry object 
from snowflake.ml.registry import Registry

# Define model name
model_name = f"MORTGAGE_LENDING_MLOPS_{VERSION_NUM}"

# Create a registry to log the model to
model_registry = Registry(session=session, 
                          database_name=DB, 
                          schema_name=SCHEMA,
                          options={"enable_monitoring": True})

### Log Baseline Model to Model Registry

Register the baseline XGBoost model in Snowflake Model Registry with metadata, training metrics, and deployment configuration.

#### **Idempotent Logging Pattern:**

Uses try/except to check if model version already exists before logging (safe to re-run).

#### **Key Registration Parameters:**

**`model=xgb_base`**: The trained baseline XGBoost classifier

**`version_name='XGB_BASE'`**: Unique identifier for this model version

**`sample_input_data`**: Sample features (100 rows) for:
- Schema inference
- Input validation
- Lineage tracking (uses Snowpark DataFrame to maintain data lineage)

**`comment`**: Documentation including hyperparameters and model description

**`target_platforms=['WAREHOUSE', 'SNOWPARK_CONTAINER_SERVICES']`**:
- Deploy to Snowflake Warehouses for batch inference
- Deploy to SPCS for real-time serving

**`enable_explainability=True`**: Enables SHAP value computation for model interpretability

#### **Attach Training Metrics:**

Logs training performance metrics (F1, Precision, Recall) to the model version for tracking and comparison against optimized models.

**Result:** Baseline model is versioned, documented, and ready for deployment with full metadata and lineage tracking.

In [ ]:
#Log the base model to the model registry (if not already there)
base_version_name = 'XGB_BASE'

try:
    #Check for existing model
    mv_base = model_registry.get_model(model_name).version(base_version_name)
    print("Found existing model version!")
except:
    print("Logging new model version...")
    #Log model to registry
    mv_base = model_registry.log_model(
        model_name=model_name,
        model=xgb_base, 
        version_name=base_version_name,
        sample_input_data = train.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"]).limit(100), #using snowpark df to maintain lineage
        comment = f"""ML model for predicting loan approval likelihood.
                    This model was trained using XGBoost classifier.
                    Hyperparameters used were:
                    max_depth={xgb_base.max_depth}, 
                    n_estimators={xgb_base.n_estimators}, 
                    learning_rate = {xgb_base.learning_rate}, 
                    algorithm = {xgb_base.booster}
                    """,
        target_platforms= ["WAREHOUSE", "SNOWPARK_CONTAINER_SERVICES"],
        options= {"enable_explainability": True}

    )
    
    #set metrics
    mv_base.set_metric(metric_name="Train_F1_Score", value=f1_base_train)
    mv_base.set_metric(metric_name="Train_Precision_Score", value=precision_base_train)
    mv_base.set_metric(metric_name="Train_Recall_score", value=recall_base_train)

### Create PROD Tag

Create a Snowflake tag to mark production-ready model versions.

---

```python
session.sql("CREATE OR REPLACE TAG PROD")
```

**What this does:**
- Creates a reusable tag named `PROD` for identifying production models
- `CREATE OR REPLACE` ensures idempotency (safe to re-run)

**Purpose:** Tags enable model lifecycle management by marking which versions are deployed to production, allowing easy identification and governance of active models.

---

**Next:** Apply the PROD tag to the baseline model version.

In [ ]:
#Create tag for PROD model
session.sql("CREATE OR REPLACE TAG PROD")

### Apply PROD Tag and Model Documentation

Add model-level documentation and tag the baseline version as the production model.

```python
m = model_registry.get_model(model_name)
m.comment = "Loan approval prediction models"
m.set_tag("PROD", base_version_name)
m.show_tags()
```

**What this does:**

**1. Get model object**: Retrieves the model family from registry

**2. Add model-level comment**: Documents the overall purpose (applies to all versions)

**3. Apply PROD tag**: Marks `XGB_BASE` version as the current production model
- Key: `"PROD"`
- Value: `base_version_name` (XGB_BASE)

**4. Display tags**: Shows all tags applied to the model

**Purpose:** Tags enable lifecycle management and governance by clearly identifying which model version is currently deployed in production. Multiple versions can exist, but only one is tagged as PROD.

**Result:** Baseline model is now marked as the active production version and can be easily identified for deployment and monitoring.

In [ ]:
#Apply prod tag 
m = model_registry.get_model(model_name)
m.comment = "Loan approval prediction models" #set model level comment
m.set_tag("PROD", base_version_name)
m.show_tags()

In [ ]:
model_registry.show_models()

In [ ]:
model_registry.get_model(model_name).show_versions()

In [ ]:
print(mv_base)
print(mv_base.show_metrics())

In [ ]:
mv_base.show_functions()

### Model Functions Output Explained

The output shows **three functions** automatically created when the model was registered with `enable_explainability=True`.


#### **Available Functions:**

**1. EXPLAIN (Table Function)**
- **Purpose**: Generate SHAP values for model interpretability
- **Inputs**: 12 features (loan purpose, temporal, financial, geographic)
- **Outputs**: 12 explanation columns (one per feature with `_explanation` suffix)
- **SHAP values**: Positive = pushes toward approval, Negative = pushes toward denial
- **Usage**: `mv_base.run(data, function_name="explain")`

**2. PREDICT (Function)**
- **Purpose**: Generate binary class predictions
- **Inputs**: Same 12 features
- **Output**: Single column `output_feature_0` with binary prediction (0 = denied, 1 = approved)
- **Usage**: `mv_base.run(data, function_name="predict")`

**3. PREDICT_PROBA (Function)**
- **Purpose**: Generate probability scores for each class
- **Inputs**: Same 12 features
- **Outputs**: Two columns with probabilities [P(denial), P(approval)]
- **Usage**: `mv_base.run(data, function_name="predict_proba")`


#### **Key Observations:**

✅ **Consistent input schema** - All functions expect the same 12 features with specific data types  
✅ **Multiple inference modes** - Choose based on use case (decision, confidence, or explanation)  
✅ **Automatic generation** - Created by Model Registry during `log_model()`  
✅ **Type safety** - Inputs and outputs have defined data types (INT8, DOUBLE, etc.)

#### **When to Use Each:**

| Function | Use Case | Example |
|----------|----------|----------|
| **PREDICT** | Final decisions | "Should we approve this loan" -> Yes/No |
| **PREDICT_PROBA** | Risk assessment, threshold tuning | "What's the confidence? Can we adjust thresholds?" |
| **EXPLAIN** | Debugging, compliance, interpretability | "Why was this loan denied? Which factors mattered most?" |

### Run Batch Inference on Test Data

Use the registered baseline model to generate predictions on the test dataset directly from Model Registry.

---

```python
reg_preds = mv_base.run(test, function_name="predict").rename(col('"output_feature_0"'), "MORTGAGE_PREDICTION")
reg_preds.show(10)
```

**What this does:**

**`mv_base.run(test, function_name="predict")`**
- Invokes the PREDICT function from Model Registry
- Runs inference on the Snowpark `test` DataFrame
- Executes in Snowflake warehouse (no data movement)
- Returns predictions in column `output_feature_0`

**`.rename(col('"output_feature_0"'), "MORTGAGE_PREDICTION")`**
- Renames the output column to a more descriptive name
- Note the escaped quotes: `'"output_feature_0"'` handles Snowflake identifier casing

**`.show(10)`**
- Displays first 10 predictions with all columns (features + prediction)

---

#### **Key Advantages of Model Registry Inference:**

✅ **Warehouse execution** - Predictions run in Snowflake, no data movement  
✅ **Version control** - Tied to specific model version (`XGB_BASE`)  
✅ **Scalable** - Handles large datasets efficiently  
✅ **Consistent** - Same preprocessing/inference pipeline as training  
✅ **Traceable** - Full lineage from features to predictions

---

**Output:** Test dataset with added `MORTGAGE_PREDICTION` column (0 = denied, 1 = approved)

**Next:** Evaluate test performance metrics to reveal overfitting.

In [ ]:
reg_preds = mv_base.run(test, function_name = "predict").rename(col('"output_feature_0"'), "MORTGAGE_PREDICTION")
reg_preds.show(10)

In [ ]:
#ds_sp_ohe = ds_sp_ohe.rename(col('"LOAN_PURPOSE_NAME_Home improvement"'), "LOAN_PURPOSE_NAME_Home_improvement")

preds_pd = reg_preds.select(["MORTGAGERESPONSE", "MORTGAGE_PREDICTION"]).to_pandas()
f1_base_test = round(f1_score(preds_pd.MORTGAGERESPONSE, preds_pd.MORTGAGE_PREDICTION),4)
precision_base_test = round(precision_score(preds_pd.MORTGAGERESPONSE, preds_pd.MORTGAGE_PREDICTION),4)
recall_base_test = round(recall_score(preds_pd.MORTGAGERESPONSE, preds_pd.MORTGAGE_PREDICTION),4)

#log metrics to model registry model
mv_base.set_metric(metric_name="Test_F1_Score", value=f1_base_test)
mv_base.set_metric(metric_name="Test_Precision_Score", value=precision_base_test)
mv_base.set_metric(metric_name="Test_Recall_score", value=recall_base_test)

print(f'F1: {f1_base_test} \nPrecision {precision_base_test} \nRecall: {recall_base_test}')

# Oh no! Our model's performance seems to have dropped off significantly from training to our test set. 
## This is evidence that our model is overfit - can we fix this with Distributed Hyperparameter Optimization??

### Prepare Data for Distributed Hyperparameter Optimization

Split train and test Snowpark DataFrames into features (X) and target (y) for use in Snowflake's distributed HPO framework.

**What this does:**

**Features (X):** All ML features, excluding:
- `MORTGAGERESPONSE` - Target variable
- `TIMESTAMP` - Not predictive
- `LOAN_ID` - Entity identifier

**Target (y):** Binary loan approval outcome (0 = denied, 1 = approved)

#### **Key Difference from Baseline Model:**

- **Baseline **: Used pandas DataFrames for local training
- **HPO **: Uses Snowpark DataFrames for distributed training in Snowflake warehouse


**Result:** Clean X/y splits ready for distributed hyperparameter optimization across multiple compute nodes.

**Next:** Configure and run distributed HPO to find optimal hyperparameters.

In [ ]:
X_train = train.drop("MORTGAGERESPONSE", "TIMESTAMP", "LOAN_ID")
y_train = train.select("MORTGAGERESPONSE")
X_test = test.drop("MORTGAGERESPONSE","TIMESTAMP", "LOAN_ID")
y_test = test.select("MORTGAGERESPONSE")

### Configure Distributed Hyperparameter Optimization (HPO)

Set up Snowflake's distributed HPO framework to find optimal hyperparameters and fix the overfitting problem observed in the baseline model.

#### **1. Prepare Dataset Connectors:**

```python
dataset_map = {
    "x_train": DataConnector.from_dataframe(X_train),
    "y_train": DataConnector.from_dataframe(y_train),
    "x_test": DataConnector.from_dataframe(X_test),
    "y_test": DataConnector.from_dataframe(y_test)
}
```

Wraps Snowpark DataFrames in `DataConnector` objects for distributed HPO consumption.

#### **2. Define Training Function:**

```python
def train_func():
    tuner_context = get_tuner_context()
    config = tuner_context.get_hyper_params()  # Get trial's hyperparameters
    dm = tuner_context.get_dataset_map()       # Get datasets
    
    model = XGBClassifier(**config, random_state=42)
    model.fit(...)
    f1_metric = f1_score(...)
    tuner_context.report(metrics={"f1_score": f1_metric}, model=model)
```

**Executed for each HPO trial** with different hyperparameter combinations.

#### **3. Configure Hyperparameter Search Space:**

```python
search_space = {
    "max_depth": tune.randint(1, 10),           # Much more conservative than baseline (50)
    "learning_rate": tune.uniform(0.01, 0.1),   # Lower range than baseline (0.75)
    "n_estimators": tune.randint(50, 100)       # More trees than baseline (3)
}
```

**Comparison to baseline:**

| Parameter | Baseline | Search Space | Why Better |
|-----------|----------|--------------|------------|
| `max_depth` | 50 | 1-10 | Prevents overfitting |
| `learning_rate` | 0.75 | 0.01-0.1 | More gradual learning |
| `n_estimators` | 3 | 50-100 | More trees = better patterns |

#### **4. Configure Tuner:**

```python
tuner = tune.Tuner(
    train_func=train_func,
    search_space=search_space,
    tuner_config=tune.TunerConfig(
        metric="f1_score",
        mode="max",                                      # Maximize F1
        search_alg=RandomSearch(random_state=101),       # Search strategy
        num_trials=8,                                    # Test 8 combinations
        max_concurrent_trials=psutil.cpu_count(logical=False)  # Parallel execution
    )
)
```

**Key settings:**
- **Optimize for**: F1 score (balance precision and recall)
- **Strategy**: Random search across parameter space
- **Trials**: 8 different hyperparameter combinations
- **Parallel**: Runs across all available CPU cores (distributed execution)

#### **🚀 What Happens Next:**

When you run `tuner.run(dataset_map)`:
1. Launches 8 parallel training jobs in Snowflake warehouse
2. Each job tests different hyperparameter combinations
3. Evaluates F1 score for each combination
4. Returns the best performing model

**Goal:** Find hyperparameters that generalize well to test data, unlike the overfit baseline model.

In [ ]:
from snowflake.ml.data import DataConnector
from snowflake.ml.modeling.tune import get_tuner_context
from snowflake.ml.modeling import tune
from entities import search_algorithm
import psutil

#Define dataset map
dataset_map = {
    "x_train": DataConnector.from_dataframe(X_train),
    "y_train": DataConnector.from_dataframe(y_train),
    "x_test": DataConnector.from_dataframe(X_test),
    "y_test": DataConnector.from_dataframe(y_test)
    }


# Define a training function, with any models you choose within it.
def train_func():
    # A context object provided by HPO API to expose data for the current HPO trial
    tuner_context = get_tuner_context()
    config = tuner_context.get_hyper_params()
    dm = tuner_context.get_dataset_map()

    model = XGBClassifier(**config, random_state=42)
    model.fit(dm["x_train"].to_pandas().sort_index(), dm["y_train"].to_pandas().sort_index())
    f1_metric = f1_score(
        dm["y_train"].to_pandas().sort_index(), model.predict(dm["x_train"].to_pandas().sort_index())
    )
    tuner_context.report(metrics={"f1_score": f1_metric}, model=model)

tuner = tune.Tuner(
    train_func=train_func,
    search_space={
        "max_depth": tune.randint(1, 10),
        "learning_rate": tune.uniform(0.01, 0.1),
        "n_estimators": tune.randint(50, 100),
    },
    tuner_config=tune.TunerConfig(
        metric="f1_score",
        mode="max",
        search_alg=search_algorithm.RandomSearch(random_state=101),
        num_trials=8, #run 8 trial runs
        max_concurrent_trials=psutil.cpu_count(logical=False) # Use all available CPUs to run distributed HPO across. GPUs can also be used here! 
    ),
)

In [ ]:
#Train several model candidates (note this may take 1-2 minutes)
tuner_results = tuner.run(dataset_map=dataset_map)

### Select Best Model from HPO

Retrieve the best performing model from the distributed hyperparameter optimization results.

```python
tuned_model = tuner_results.best_model
```

Extracts the XGBoost model with the hyperparameters that achieved the highest F1 score across all 8 trials.

#### **✅ Optimized Hyperparameters Found:**

| Parameter | Baseline (Overfit) | Optimized (HPO) | Improvement |
|-----------|-------------------|-----------------|-------------|
| `max_depth` | 50 | **7** | ✅ Much shallower (prevents overfitting) |
| `learning_rate` | 0.75 | **0.043** | ✅ More gradual learning |
| `n_estimators` | 3 | **93** | ✅ More trees for better patterns |
| `random_state` | None | **42** | ✅ Reproducible results |

#### **Key Observations:**

**`max_depth=7`** (vs baseline 50)
- Prevents trees from memorizing training data
- Better generalization to unseen data

**`learning_rate=0.043`** (vs baseline 0.75)
- Smaller, more conservative updates
- Reduces risk of overshooting optimal weights

**`n_estimators=93`** (vs baseline 3)
- Many more trees to learn complex patterns
- Ensemble approach improves robustness

#### **Result:**

A well-tuned XGBoost classifier that should demonstrate:
- ✅ **Modest training accuracy** (not perfect - doesn't memorize)
- ✅ **Strong test accuracy** (good generalization)
- ✅ **Minimal overfitting** (small train/test performance gap)

**Next:** Evaluate this optimized model on train and test sets to confirm improved generalization compared to the baseline.

In [ ]:
#Select best model results and inspect configuration
tuned_model = tuner_results.best_model
tuned_model

In [ ]:
#Generate predictions
xgb_opt_preds = tuned_model.predict(train_pd.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"],axis=1))

#Generate performance metrics
f1_opt_train = round(f1_score(train_pd.MORTGAGERESPONSE, xgb_opt_preds),4)
precision_opt_train = round(precision_score(train_pd.MORTGAGERESPONSE, xgb_opt_preds),4)
recall_opt_train = round(recall_score(train_pd.MORTGAGERESPONSE, xgb_opt_preds),4)

print(f'Train Results: \nF1: {f1_opt_train} \nPrecision {precision_opt_train} \nRecall: {recall_opt_train}')

In [ ]:
#Generate test predictions
xgb_opt_preds_test = tuned_model.predict(test_pd.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"],axis=1))

#Generate performance metrics on test data
f1_opt_test = round(f1_score(test_pd.MORTGAGERESPONSE, xgb_opt_preds_test),4)
precision_opt_test = round(precision_score(test_pd.MORTGAGERESPONSE, xgb_opt_preds_test),4)
recall_opt_test = round(recall_score(test_pd.MORTGAGERESPONSE, xgb_opt_preds_test),4)

print(f'Test Results: \nF1: {f1_opt_test} \nPrecision {precision_opt_test} \nRecall: {recall_opt_test}')

# Here we see the HPO model has a more modest train accuracy than our base model - but the peformance doesn't drop off during testing

In [ ]:
#Log the optimized model to the model registry (if not already there)
optimized_version_name = 'XGB_Optimized'

try:
    #Check for existing model
    mv_opt = model_registry.get_model(model_name).version(optimized_version_name)
    print("Found existing model version!")
except:
    #Log model to registry
    print("Logging new model version...")
    mv_opt = model_registry.log_model(
        model_name=model_name,
        model=tuned_model, 
        version_name=optimized_version_name,
        sample_input_data = train.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"]).limit(100),
        comment = f"""HPO ML model for predicting loan approval likelihood.
            This model was trained using XGBoost classifier.
            Optimized hyperparameters used were:
            max_depth={tuned_model.max_depth}, 
            n_estimators={tuned_model.n_estimators}, 
            learning_rate = {tuned_model.learning_rate}, 
            """,
        target_platforms= ["WAREHOUSE", "SNOWPARK_CONTAINER_SERVICES"],
        options= {"enable_explainability": True}

        

    )
    #Set metrics
    mv_opt.set_metric(metric_name="Train_F1_Score", value=f1_opt_train)
    mv_opt.set_metric(metric_name="Train_Precision_Score", value=precision_opt_train)
    mv_opt.set_metric(metric_name="Train_Recall_score", value=recall_opt_train)

    mv_opt.set_metric(metric_name="Test_F1_Score", value=f1_opt_test)
    mv_opt.set_metric(metric_name="Test_Precision_Score", value=precision_opt_test)
    mv_opt.set_metric(metric_name="Test_Recall_score", value=recall_opt_test)

In [ ]:
#Here we see the BASE version is our default version
model_registry.get_model(model_name).default

In [ ]:
#Now we'll set the optimized model to be the default model version going forward
model_registry.get_model(model_name).default = optimized_version_name

In [ ]:
#Now we see our optimized version we have now recently promoted to our DEFAULT model version
model_registry.get_model(model_name).default

In [ ]:
#we'll now update the PROD tagged model to be the optimized model version rather than our overfit base version
m.unset_tag("PROD")
m.set_tag("PROD", optimized_version_name)
m.show_tags()

## Model Explainability with SHAP Values

Now that both baseline and optimized models are deployed, use Snowflake's built-in explainability features to understand how input features impact model predictions.

### **What is Model Explainability?**

**SHAP (SHapley Additive exPlanations)** values quantify each feature's contribution to individual predictions:
- **Positive values**: Feature pushes prediction toward approval
- **Negative values**: Feature pushes prediction toward denial
- **Magnitude**: Strength of the feature's impact


### **Generate SHAP Values:**

```python
# Create sample of 2,500 test records
test_pd_sample = test_pd.rename(columns=rename_dict).sample(n=2500, random_state=100).reset_index(drop=True)

# Compute SHAP values for baseline model
base_shap_pd = mv_base.run(test_pd_sample, function_name="explain")

# Compute SHAP values for optimized model
opt_shap_pd = mv_opt.run(test_pd_sample, function_name="explain")
```

**What this does:**

**1. Sample test data**: 2,500 records for efficient SHAP computation (SHAP is computationally expensive)

**2. Rename columns**: Apply `rename_dict` to match model's expected input schema

**3. Generate SHAP values**: Call `explain` function for both models
   - `base_shap_pd`: Explanations from overfit baseline model
   - `opt_shap_pd`: Explanations from optimized HPO model

### **Output Structure:**

Each SHAP DataFrame contains **12 explanation columns** (one per feature):
- LOAN_PURPOSE_NAME_HOME_IMPROVEMENT_explanation
- LOAN_PURPOSE_NAME_HOME_PURCHASE_explanation
- LOAN_PURPOSE_NAME_REFINANCING_explanation
- MONTH_explanation
- DAY_OF_YEAR_explanation
- DOTW_explanation
- LOAN_AMOUNT_explanation
- INCOME_explanation
- INCOME_LOAN_RATIO_explanation
- MEAN_COUNTY_INCOME_explanation
- HIGH_INCOME_FLAG_explanation
- AVG_THIRTY_DAY_LOAN_als differences in how models make decisions

### **Key Benefits:**

✅ **Built-in to Model Registry** - No additional SHAP library configuration  
✅ **Scalable** - Computes in Snowflake warehouse  
✅ **Versioned** - Each model version has consistent explainability  
✅ **Compliance-ready** - Explain individual predictions for regulatory requirements


**Next:** Visualize SHAP values to understand feature importance and model behavior.


In [ ]:
#create a sample of 1000 records
test_pd_sample=test_pd.rename(columns=rename_dict).sample(n=2500, random_state = 100).reset_index(drop=True)

#Compute shapley values for each model
base_shap_pd = mv_base.run(test_pd_sample, function_name="explain")
opt_shap_pd = mv_opt.run(test_pd_sample, function_name="explain")

### Visualize SHAP Values with Built-in Snowflake Functions

Use Snowflake's built-in visualization functions to interpret how features influence the baseline model's predictions.

#### **Prepare Feature DataFrame:**

```python
feat_df = test_pd_sample.drop(["MORTGAGERESPONSE", "TIMESTAMP", "LOAN_ID"], axis=1)
```

Extracts only the 12 ML features (excludes target and identifiers) for visualization.

#### **Generate Influence Sensitivity Plot:**

```python
explain_visualize.plot_influence_sensitivity(base_shap_pd, feat_df, figsize=(1500, 500))
```

**What this creates:**
- **SHAP dependence scatter plot** showing the relationship between feature values and their SHAP contributions
- **X-axis**: Feature values (e.g., INCOME amounts)
- **Y-axis**: SHAP values (positive = pushes toward approval, negative = toward denial)
- **Interactive**: In Snowflake Notebooks, includes dropdown to select which feature to visualize

**Purpose:** Understand how changes in feature values influence predictions (e.g., "Does higher income always increase approval probability?")

#### **Additional Built-in Visualizations (Optional):**

**1. Force Plot** - Single prediction explanation:
```python
explain_visualize.plot_force(base_shap_pd.iloc[0], feat_df.iloc[0], figsize=(1500, 500))
```
- Shows how each feature pushes one specific prediction higher or lower
- Red arrows = positive contributions (toward approval)
- Blue arrows = negative contributions (toward denial)
- Visualizes the path from base prediction to final prediction

**2. Violin Plot** - Feature importance distribution:
```python
explain_visualize.plot_violin(base_shap_pd, feat_df, figsize=(1400, 100))
```
- Shows distribution and range of SHAP values for each feature
- Sorted by absolute mean SHAP value (most influential features at top)
- Reveals which features have the most significant impact overall

#### **Key Benefits:**

✅ **Built-in to Snowflake** - No external SHAP library configuration  
✅ **Interactive** (in Snowflake Notebooks) - Dropdown feature selection  
✅ **Multiple perspectives** - Dependence, force, and distribution views  
✅ **Production-ready** - Scalable visualizations for model debugging

#### **When to Use Each:**

| Visualization | Use Case |
|---------------|----------|
| **Influence Sensitivity** | Understand feature-prediction relationships across dataset |
| **Force Plot** | Explain a single prediction to a stakeholder |
| **Violin Plot** | Identify most important features overall |

**Result:** Visual insights into which features drive the baseline model's decisions and how they interact with predictions.

In [ ]:
from snowflake.ml.monitoring import explain_visualize

feat_df=test_pd_sample.drop(["MORTGAGERESPONSE","TIMESTAMP", "LOAN_ID"],axis=1)

explain_visualize.plot_influence_sensitivity(base_shap_pd, feat_df, figsize=(1500, 500))

#Optionally test out other built-in functionality 
# explain_visualize.plot_force(base_shap_pd.iloc[0], feat_df.iloc[0], figsize=(1500, 500))
# explain_visualize.plot_violin(base_shap_pd, feat_df, figsize=(1400, 100))

### In addition to built-in visualization capabilities you can always use open source packages like shap for additional visualizations

In [ ]:
import shap 

shap.summary_plot(np.array(base_shap_pd.astype(float)), 
                  test_pd_sample.drop(["LOAN_ID","MORTGAGERESPONSE", "TIMESTAMP"], axis=1), 
                  feature_names = test_pd_sample.drop(["LOAN_ID","MORTGAGERESPONSE", "TIMESTAMP"], axis=1).columns)

In [ ]:
shap.summary_plot(np.array(opt_shap_pd.astype(float)), 
                  test_pd_sample.drop(["LOAN_ID","MORTGAGERESPONSE", "TIMESTAMP"], axis=1), 
                  feature_names = test_pd_sample.drop(["LOAN_ID","MORTGAGERESPONSE", "TIMESTAMP"], axis=1).columns)

In [ ]:
#Merge shap vals and actual vals together for easier plotting below
all_shap_base = test_pd_sample.merge(base_shap_pd, right_index=True, left_index=True, how='outer')
all_shap_opt = test_pd_sample.merge(opt_shap_pd, right_index=True, left_index=True, how='outer')

### Compare INCOME Feature Impact: Base vs Optimized Models

Create side-by-side scatter plots to visualize how the INCOME feature influences predictions differently in the baseline (overfit) and optimized models.

#### **Data Preparation:**

```python
asb_filtered = all_shap_base[(all_shap_base.INCOME>0) & (all_shap_base.INCOME<250000)]
aso_filtered = all_shap_opt[(all_shap_opt.INCOME>0) & (all_shap_opt.INCOME<250000)]
```

**Filter outliers** to focus on the main income range ($0-$250K) for clearer visualization.

- `all_shap_base`: SHAP explanations from baseline model
- `all_shap_opt`: SHAP explanations from optimized model

#### **Visualization:**

```python
sns.scatterplot()  # Plot SHAP values for each income level
sns.regplot()      # Add regression line to show overall trend
```

**Left Plot (Base Model):**
- Blue scatter points with red trend line
- Shows INCOME vs INCOME_explanation relationship for overfit model

**Right Plot (Optimized Model):**
- Orange scatter points with blue trend line
- Shows same relationship for HPO-tuned model

#### **What to Look For:**

✅ **Scatter tightness** - How consistent are SHAP values for similar incomes?  
✅ **Trend line slope** - Does higher income lead to higher approval influence?  
✅ **Y-axis range** - How extreme are the SHAP values?  
✅ **Outliers** - Are there many unusual cases?


**Goal:** Demonstrate that hyperparameter optimization produces more stable, interpretable feature impacts compared to the overfit baseline.


sns.scatterplot()  # Plot SHAP values for each income level
sns.regplot()      # Add regression line to show overall trendeter optimization in creating generalizable, production-ready predictions.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#filter data down to strip outliers
asb_filtered = all_shap_base[(all_shap_base.INCOME>0) & (all_shap_base.INCOME<250000)]
aso_filtered = all_shap_opt[(all_shap_opt.INCOME>0) & (all_shap_opt.INCOME<250000)]

# Set up the figure
fig, axes = plt.subplots(1, 2, figsize=(10, 6))
fig.suptitle("INCOME EXPLANATION")
# Plot side-by-side boxplots
sns.scatterplot(data = asb_filtered, x ='INCOME', y = 'INCOME_explanation', ax=axes[0])
sns.regplot(data = asb_filtered, x ="INCOME", y = 'INCOME_explanation', scatter=False, color='red', line_kws={"lw":2},ci =100, lowess=False, ax =axes[0])

axes[0].set_title('Base Model')
sns.scatterplot(data = aso_filtered, x ='INCOME', y = 'INCOME_explanation',color = "orange", ax = axes[1])
sns.regplot(data = aso_filtered, x ="INCOME", y = 'INCOME_explanation', scatter=False, color='blue', line_kws={"lw":2},ci =100, lowess=False, ax =axes[1])
axes[1].set_title('Opt Model')

# Customize and show the plot
for ax in axes:
    ax.set_xlabel("Income")
    ax.set_ylabel("Influence")
plt.tight_layout()
plt.show()



### Results: Dramatic Improvement in Model Stability

Comparing SHAP value distributions reveals the success of hyperparameter optimization in creating generalizable, production-ready predictions.

#### **🔴 Base Model (Left Plot - Blue/Red)**

**Observations:**

**Extreme Variance**
- SHAP values range from **-1.5 to +1.0** (2.5 unit spread)
- Same income level shows wildly different impacts
- Example: $100K income → SHAP ranges from -1.0 to +0.8

**Weak Positive Trend**
- Red line shows slight upward slope
- But trend is **overwhelmed by scatter**
- Poor predictive consistency

**Unrealistic Patterns**
- Many low-income cases have positive SHAP (pushes toward approval)
- Many high-income cases have negative SHAP (pushes toward denial)
- **Doesn't align with business logic**

**🚨 Diagnosis:** Classic overfitting - model memorized training examples rather than learning generalizable patterns.

#### **🟢 Optimized Model (Right Plot - Orange/Blue)**

**Observations:**

**Dramatically Reduced Variance**
- SHAP values range from **-0.15 to +0.10** (0.25 unit spread)
- **10x reduction in variance** compared to baseline
- Consistent treatment of similar income levels

**Nearly Flat Trend**
- Blue line is almost horizontal
- Income has **modest, stable influence**
- Reflects sophisticated feature interactions

**Few Outliers**
- Only 2-3 points exceed ±0.10
- Most values cluster tightly around zero

**✅ Diagnosis:** Well-generalized model with stable, interpretable patterns.


#### **📊 Key Comparisons**

| Metric | Base Model | Optimized Model | Improvement |
|--------|------------|-----------------|-------------|
| **SHAP Range** | 2.5 units | 0.25 units | **10x reduction** |
| **Consistency** | Chaotic | Stable | **Predictable** |
| **Outliers** | Many extremes | Few exceptions | **Robust** |
| **Business Logic** | Violated | Aligned | **Trustworthy** |


#### **💡 Why is the Optimized Trend Nearly Flat?**

**This is actually a GOOD sign!**

1. **Feature Interactions**: Model learned that income's effect is **contextual**
   - High income + large loan ≠ High income + small loan
   - Income matters differently based on other factors

2. **Engineered Features Handle the Heavy Lifting**:
   - `INCOME_LOAN_RATIO` captures relative affordability
   - `HIGH_INCOME_FLAG` captures threshold effects
   - `MEAN_COUNTY_INCOME` provides geographic context
   - Raw INCOME becomes less critical

3. **Ensemble Learning** (93 trees): No single feature dominates, leading to balanced, robust predictions


#### **🎯 Conclusion**

**Hyperparameter optimization successfully fixed the overfitting problem:**

- ✅ **Stable predictions** - Similar applicants get similar treatment
- ✅ **Fair decisions** - Consistent feature impacts
- ✅ **Production-ready** - Predictable, explainable behavior
- ✅ **Better generalization** - Learns patterns, not memorizes examples

**This visualization proves the optimized model is ready for deployment.**


In [ ]:
#filter data down to strip outliers
asb_filtered = all_shap_base[all_shap_base.LOAN_AMOUNT<2000000]
aso_filtered = all_shap_opt[all_shap_opt.LOAN_AMOUNT<2000000]


# Set up the figure
fig, axes = plt.subplots(1, 2, figsize=(10, 6))
fig.suptitle("LOAN_AMOUNT EXPLANATION")
# Plot side-by-side boxplots
sns.scatterplot(data = asb_filtered, x ='LOAN_AMOUNT', y = 'LOAN_AMOUNT_explanation', ax=axes[0])
sns.regplot(data = asb_filtered, x ="LOAN_AMOUNT", y = 'LOAN_AMOUNT_explanation', scatter=False, color='red', line_kws={"lw":2},ci =100, lowess=True, ax =axes[0])
axes[0].set_title('Base Model')

sns.scatterplot(data = aso_filtered, x ='LOAN_AMOUNT', y = 'LOAN_AMOUNT_explanation',color = "orange", ax = axes[1])
sns.regplot(data = aso_filtered, x ="LOAN_AMOUNT", y = 'LOAN_AMOUNT_explanation', scatter=False, color='blue', line_kws={"lw":2},ci =100, lowess=True, ax =axes[1])
axes[1].set_title('Opt Model')

# Customize and show the plot
for ax in axes:
    ax.set_xlabel("LOAN_AMOUNT")
    ax.set_ylabel("Influence")
    # ax.set_xlim((0,10000))
plt.tight_layout()
plt.show()


In [ ]:
# Set up the figure
fig, axes = plt.subplots(1, 2, figsize=(10, 6))
fig.suptitle("HOME PURCHASE LOAN EXPLANATION")
# Plot side-by-side boxplots
sns.boxplot(data = all_shap_base, x ='LOAN_PURPOSE_NAME_HOME_PURCHASE', y = 'LOAN_PURPOSE_NAME_HOME_PURCHASE_explanation',
            hue='LOAN_PURPOSE_NAME_HOME_PURCHASE', width=0.8, ax=axes[0])
axes[0].set_title('Base Model')
sns.boxplot(data = all_shap_opt, x ='LOAN_PURPOSE_NAME_HOME_PURCHASE', y = 'LOAN_PURPOSE_NAME_HOME_PURCHASE_explanation',
            hue='LOAN_PURPOSE_NAME_HOME_PURCHASE', width=0.4, ax = axes[1])
axes[1].set_title('Opt Model')

# Customize and show the plot
for ax in axes:
    ax.set_xlabel("Home PURCHASE Loan (1 = True)")
    ax.set_ylabel("Influence")
    ax.legend(loc='upper right')

plt.show()


In [ ]:
# Set up the figure
fig, axes = plt.subplots(1, 2, figsize=(10, 6))
fig.suptitle("HOME IMPROVEMENT LOAN EXPLANATION")
# Plot side-by-side boxplots
sns.boxplot(data = all_shap_base, x ='LOAN_PURPOSE_NAME_HOME_IMPROVEMENT', y = 'LOAN_PURPOSE_NAME_HOME_IMPROVEMENT_explanation',
            hue='LOAN_PURPOSE_NAME_HOME_IMPROVEMENT', width=0.8, ax=axes[0])
axes[0].set_title('Base Model')
sns.boxplot(data = all_shap_opt, x ='LOAN_PURPOSE_NAME_HOME_IMPROVEMENT', y = 'LOAN_PURPOSE_NAME_HOME_IMPROVEMENT_explanation',
            hue='LOAN_PURPOSE_NAME_HOME_IMPROVEMENT', width=0.4, ax = axes[1])
axes[1].set_title('Opt Model')

# Customize and show the plot
for ax in axes:
    ax.set_xlabel("Home Improvement Loan (1 = True)")
    ax.set_ylabel("Influence")
    ax.legend(loc='upper right')

plt.show()


# Model Monitoring setup

### Prepare Data for Model Monitoring

Save training and test datasets as permanent Snowflake tables to enable model monitoring and drift detection.

```python
train.write.save_as_table(f"DEMO_MORTGAGE_LENDING_TRAIN_{VERSION_NUM}", mode="overwrite")
test.write.save_as_table(f"DEMO_MORTGAGE_LENDING_TEST_{VERSION_NUM}", mode="overwrite")
```

**What this does:**

**Persists Snowpark DataFrames as tables:**
- `train` → `DEMO_MORTGAGE_LENDING_TRAIN_{VERSION_NUM}`
- `test` → `DEMO_MORTGAGE_LENDING_TEST_{VERSION_NUM}`

**Parameters:**
- **Table names**: Versioned with `VERSION_NUM` for tracking
- **`mode="overwrite"`**: Replaces table if it already exists (idempotent)

#### **Why Save These Tables?**

**1. Model Monitoring Requirements**
- Snowflake Model Monitor needs **reference data** to establish baseline distributions
- Training data serves as the "expected" distribution
- Test data can be used for initial validation

**2. Drift Detection**
- Future production data will be compared against these reference tables
- Detect if feature distributions change over time
- Alert when model performance degrades

**3. Reproducibility**
- Permanent record of exact data used for training/testing
- Version control through `VERSION_NUM`
- Can recreate monitoring setup at any time

**4. Lineage & Governance**
- Links models to their training data
- Audit trail for compliance
- Enables investigation if issues arise

#### **What Happens Next:**

These tables will be used to:
1. ✅ **Create Model Monitor** - Reference for baseline metrics
2. ✅ **Configure drift detection** - Compare production data against baseline
3. ✅ **Track performance** - Monitor model accuracy over time
4. ✅ **Trigger alerts** - Notify when retraining is needed

**Result:** Training and test datasets are now permanently stored and ready for model monitoring configuration.

In [ ]:
train.write.save_as_table(f"DEMO_MORTGAGE_LENDING_TRAIN_{VERSION_NUM}", mode="overwrite")
test.write.save_as_table(f"DEMO_MORTGAGE_LENDING_TEST_{VERSION_NUM}", mode="overwrite")

In [ ]:
session.sql("CREATE stage IF NOT EXISTS ML_STAGE").collect()

### Define and Register Stored Procedure for Model Inference

Create a reusable stored procedure (SPROC) that retrieves a model from Model Registry and runs batch inference on any specified table.

---

#### **Stored Procedure Function:**

```python
def demo_inference_sproc(session: snowpark.Session, table_name: str, modelname: str, modelversion: str) -> str:
```

**Parameters:**
- `session`: Snowpark session (automatically provided by Snowflake)
- `table_name`: Name of the table containing data for inference
- `modelname`: Model name in Model Registry
- `modelversion`: Specific model version to use

**Returns:** `"Success"` message upon completion

---

#### **What the SPROC Does:**

**1. Retrieve Model from Registry**
```python
reg = Registry(session=session)
m = reg.get_model(model_name)
mv = m.version(modelversion)
```
Fetches the specified model version from Snowflake Model Registry

**2. Load Input Data**
```python
df = session.table(input_table_name)
```
Reads the target table as a Snowpark DataFrame

**3. Run Batch Inference**
```python
results = mv.run(df, function_name="predict").select("LOAN_ID", '"output_feature_0"').withColumnRenamed('"output_feature_0"', pred_col)
```
- Calls the model's `predict` function
- Selects loan ID and prediction
- Renames output to versioned column name (e.g., `XGB_BASE_PREDICTION`)

**4. Join Predictions with Original Data**
```python
final = df.join(results, on="LOAN_ID", how="full")
```
Combines original features with predictions

**5. Write Results Back**
```python
final.write.save_as_table(table_name, mode='overwrite', enable_schema_evolution=True)
```
- Overwrites original table with predictions added
- `enable_schema_evolution=True`: Automatically adds new prediction column

---

#### **Register the SPROC:**

```python
session.sproc.register(
    func=demo_inference_sproc,
    name="model_inference_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['joblib', 'snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=StringType()
)
```

**Configuration:**
- **`name`**: `model_inference_sproc` - How to call it in Snowflake
- **`replace=True`**: Overwrites if already exists (idempotent)
- **`is_permanent=True`**: Persists beyond session (survives restarts)
- **`stage_location`**: `@ML_STAGE` - Where Python packages are stored
- **`packages`**: Required dependencies for model execution
- **`return_type`**: Returns a string

---

#### **🎯 Why Use a Stored Procedure?**

**1. Production Deployment**
- ✅ **Scalable**: Runs on Snowflake compute (no client resources)
- ✅ **Scheduled**: Can be called by Snowflake Tasks for automation
- ✅ **Centralized**: Logic lives in Snowflake, not external scripts

**2. Reusability**
- ✅ **Parameterized**: Works with any table, model, or version
- ✅ **Versioned**: Can switch models without code changes
- ✅ **Shareable**: Available to all users with permissions

**3. Model Registry Integration**
- ✅ **Always uses latest**: Automatically fetches model from registry
- ✅ **Version control**: Specify exact version for reproducibility
- ✅ **Lineage tracking**: Maintains connection between models and predictions

---

#### **How to Use:**

```sql
-- Call the SPROC to run inference
CALL model_inference_sproc(
    'DEMO_MORTGAGE_LENDING_TEST_0',  -- table_name
    'MORTGAGE_LENDING_MLOPS_0',      -- modelname
    'XGB_BASE'                       -- modelversion
);
```

**Result:** Table is updated with a new prediction column (e.g., `XGB_BASE_PREDICTION`)

---

#### **Next Steps:**

After registration, this SPROC can be:
- ✅ Called manually for ad-hoc inference
- ✅ Scheduled with Snowflake Tasks for batch processing
- ✅ Integrated into data pipelines
- ✅ Used for A/B testing different model versions

---

**This SPROC provides a production-ready, scalable way to deploy models from Model Registry for batch inference!**

In [ ]:
from snowflake import snowpark

def demo_inference_sproc(session: snowpark.Session, table_name: str, modelname: str, modelversion: str) -> str:

    reg = Registry(session=session)
    m = reg.get_model(model_name)  # Fetch the model using the registry
    mv = m.version(modelversion)
    
    input_table_name=table_name
    pred_col = f'{modelversion}_PREDICTION'

    # Read the input table to a dataframe
    df = session.table(input_table_name)
    results = mv.run(df, function_name="predict").select("LOAN_ID",'"output_feature_0"').withColumnRenamed('"output_feature_0"', pred_col)
    # 'results' is the output DataFrame with predictions

    final = df.join(results, on="LOAN_ID", how="full")
    # Write results back to Snowflake table
    final.write.save_as_table(table_name, mode='overwrite',enable_schema_evolution=True)

    return "Success"

# Register the stored procedure
session.sproc.register(
    func=demo_inference_sproc,
    name="model_inference_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['joblib', 'snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=StringType()
)


In [ ]:
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}','{{model_name}}', '{{base_version_name}}');

In [ ]:
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}','{{model_name}}', '{{base_version_name}}');

In [ ]:
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}','{{model_name}}', '{{optimized_version_name}}');

In [ ]:
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}','{{model_name}}', '{{optimized_version_name}}');

### Run Batch Inference: Both Models on Train & Test Data

Execute the stored procedure four times to generate predictions from both baseline and optimized models on training and test datasets.

---

#### **Execution Sequence:**

**1. Baseline Model → Training Data**
```sql
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}','{{model_name}}', '{{base_version_name}}');
```
Adds `XGB_BASE_PREDICTION` column to training table

**2. Baseline Model → Test Data**
```sql
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}','{{model_name}}', '{{base_version_name}}');
```
Adds `XGB_BASE_PREDICTION` column to test table

**3. Optimized Model → Training Data**
```sql
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}','{{model_name}}', '{{optimized_version_name}}');
```
Adds `XGB_OPTIMIZED_PREDICTION` column to training table

**4. Optimized Model → Test Data**
```sql
CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}','{{model_name}}', '{{optimized_version_name}}');
```
Adds `XGB_OPTIMIZED_PREDICTION` column to test table


CALL model_inference_sproc('DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}','{{model_name}}', '{{base_version_name}}');ns |
|-------|---------------------|------------------|
| Baseline (Overfit) | ✅ | ✅ |
| Optimized (HPO) | ✅ | ✅ |

**Enables:**
1. ✅ **Performance comparison** - Calculate metrics for both models side-by-side
2. ✅ **Overfitting detection** - Compare train vs test performance for each model
3. ✅ **Model monitoring setup** - Reference data with predictions for drift detection
4. ✅ **A/B testing** - Both predictions available for evaluation


#### **📈 Expected Outcomes:**

**Baseline Model (XGB_BASE):**
- High training accuracy (overfitting)
- Lower test accuracy (poor generalization)
- Large train/test performance gap

**Optimized Model (XGB_OPTIMIZED):**
- Modest training accuracy (not memorizing)
- Strong test accuracy (good generalization)
- Small train/test performance gap

#### **Next Steps:**

With predictions generated, you can now:
- 📊 Calculate and compare F1, precision, recall metrics
- 🎨 Create confusion matrices and ROC curves
- 📉 Visualize train/test performance differences
- 🔍 Set up Model Monitors for both model versions
- ✅ Demonstrate HPO success with concrete metrics

---

**Result:** Complete inference results from both models ready for comprehensive performance analysis and monitoring setup.

In [ ]:
select TIMESTAMP, LOAN_ID, INCOME, LOAN_AMOUNT, XGB_BASE_PREDICTION, XGB_OPTIMIZED_PREDICTION, MORTGAGERESPONSE 
FROM DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}} 
limit 20

## Now that our models have been deployed and we have run inference - lets set up ML Observability!

## Set Up Model Monitoring

Configure monitoring to track performance, detect drift, and ensure production reliability.

---

### **Prepare Segmentation Column**

Add a human-readable `LOAN_PURPOSE` column to enable drill-down analysis by loan type.

---

```sql
ALTER TABLE DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}
ADD COLUMN IF NOT EXISTS LOAN_PURPOSE VARCHAR(50);

UPDATE DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}
SET LOAN_PURPOSE = CASE
    WHEN LOAN_PURPOSE_NAME_HOME_IMPROVEMENT = 1 THEN 'HOME_IMPROVEMENT'
    WHEN LOAN_PURPOSE_NAME_HOME_PURCHASE = 1 THEN 'HOME_PURCHASE'
    WHEN LOAN_PURPOSE_NAME_REFINANCING = 1 THEN 'REFINANCING'
    ELSE 'OTHER'
END;
```

**What this does:**
- Converts one-hot encoded binary flags to categorical labels
- Enables segmented monitoring (e.g., drift detection by loan type)
- Improves dashboard readability and root cause analysis

**Result:** Test table now includes `LOAN_PURPOSE` column for segment-based monitoring.

---

**Next:** Create Model Monitors for both baseline and optimized models.

In [ ]:
ALTER TABLE DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}
ADD COLUMN IF NOT EXISTS LOAN_PURPOSE VARCHAR(50);


UPDATE DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}
SET LOAN_PURPOSE = CASE
    WHEN LOAN_PURPOSE_NAME_HOME_IMPROVEMENT = 1 THEN 'HOME_IMPROVEMENT'
    WHEN LOAN_PURPOSE_NAME_HOME_PURCHASE = 1 THEN 'HOME_PURCHASE'
    WHEN LOAN_PURPOSE_NAME_REFINANCING = 1 THEN 'REFINANCING'
    ELSE 'OTHER'
END;

In [ ]:
ALTER TABLE DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}
ADD COLUMN IF NOT EXISTS LOAN_PURPOSE VARCHAR(50);


UPDATE DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}
SET LOAN_PURPOSE = CASE
    WHEN LOAN_PURPOSE_NAME_HOME_IMPROVEMENT = 1 THEN 'HOME_IMPROVEMENT'
    WHEN LOAN_PURPOSE_NAME_HOME_PURCHASE = 1 THEN 'HOME_PURCHASE'
    WHEN LOAN_PURPOSE_NAME_REFINANCING = 1 THEN 'REFINANCING'
    ELSE 'OTHER'
END;

In [ ]:
SELECT LOAN_PURPOSE_NAME_HOME_PURCHASE, LOAN_PURPOSE_NAME_HOME_IMPROVEMENT, LOAN_PURPOSE_NAME_REFINANCING, LOAN_PURPOSE FROM DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}} limit 10;

### Create Model Monitor for Baseline Model

Set up a Model Monitor to track performance, detect drift, and monitor data quality for the baseline (overfit) model.

#### **Configuration Parameters:**

**Model Reference:**
- **`MODEL`**: `MORTGAGE_LENDING_MLOPS_0` - Model from registry
- **`VERSION`**: `XGB_BASE` - Baseline model version
- **`FUNCTION`**: `predict` - Model function to monitor

**Data Sources:**
- **`SOURCE`**: Test table containing inference results (current predictions)
- **`BASELINE`**: Training table serving as reference distribution (expected behavior)

**Column Mapping:**
- **`TIMESTAMP_COLUMN`**: `TIMESTAMP` - For time-series drift analysis
- **`PREDICTION_CLASS_COLUMNS`**: `XGB_BASE_PREDICTION` - Model's predicted values
- **`ACTUAL_CLASS_COLUMNS`**: `MORTGAGERESPONSE` - Ground truth labels
- **`ID_COLUMNS`**: `LOAN_ID` - Unique record identifier

**Segmentation:**
- **`SEGMENT_COLUMNS`**: `LOAN_PURPOSE` - Enable drill-down by loan type

**Compute & Scheduling:**
- **`WAREHOUSE`**: `APP_WH` - Compute for monitoring queries
- **`REFRESH_INTERVAL`**: `12 hours` - How often to update metrics
- **`AGGREGATION_WINDOW`**: `1 day` - Time window for aggregating metrics

---

#### **What This Monitor Tracks:**

**1. Performance Metrics**
- Accuracy, F1 score, precision, recall over time
- Compare predictions vs actual labels

**2. Data Drift**
- Feature distribution changes (vs baseline training data)
- Detect shifts in INCOME, LOAN_AMOUNT, etc.

**3. Prediction Drift**
- Changes in prediction distribution
- Alert if prediction patterns differ from training

**4. Segmented Analysis**
- Monitor drift separately for:
  - HOME_PURCHASE loans
  - REFINANCING loans  
  - HOME_IMPROVEMENT loans

---

#### **How It Works:**
Every 12 hours:
1. Query SOURCE table (test data with predictions)
2. Compare against BASELINE table (training data distribution)
3. Calculate drift metrics for 1-day windows
4. Store results for querying/alerting.

In [ ]:
CREATE OR REPLACE MODEL MONITOR MORTGAGE_LENDING_BASE_MODEL_MONITOR
WITH
    MODEL={{model_name}}
    VERSION={{base_version_name}}
    FUNCTION=predict
    SOURCE=DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}
    BASELINE=DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}
    TIMESTAMP_COLUMN=TIMESTAMP
    PREDICTION_CLASS_COLUMNS=(XGB_BASE_PREDICTION)  
    ACTUAL_CLASS_COLUMNS=(MORTGAGERESPONSE)
    ID_COLUMNS=(LOAN_ID)
    SEGMENT_COLUMNS = ('LOAN_PURPOSE')
    WAREHOUSE={{COMPUTE_WAREHOUSE}}
    REFRESH_INTERVAL='12 hours'
    AGGREGATION_WINDOW='1 day';

### Create Model Monitor for Optimized Model

Set up monitoring for the optimized (HPO-tuned) model to track its performance and drift.

**Key difference from baseline monitor:**
- **`VERSION`**: `XGB_OPTIMIZED` - Monitors the HPO-tuned model
- **`PREDICTION_CLASS_COLUMNS`**: `XGB_OPTIMIZED_PREDICTION` - Uses optimized model's predictions

**Configuration:** Same data sources, refresh schedule, and segmentation as baseline monitor

**Result:** Both models now have active monitoring for performance comparison and drift detection.

In [ ]:
CREATE OR REPLACE MODEL MONITOR MORTGAGE_LENDING_OPTIMIZED_MODEL_MONITOR
WITH
    MODEL={{model_name}}
    VERSION={{optimized_version_name}}
    FUNCTION=predict
    SOURCE=DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}
    BASELINE=DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}
    TIMESTAMP_COLUMN=TIMESTAMP
    PREDICTION_CLASS_COLUMNS=(XGB_OPTIMIZED_PREDICTION)  
    ACTUAL_CLASS_COLUMNS=(MORTGAGERESPONSE)
    ID_COLUMNS=(LOAN_ID)
    SEGMENT_COLUMNS = ('LOAN_PURPOSE')
    WAREHOUSE={{COMPUTE_WAREHOUSE}}
    REFRESH_INTERVAL='12 hours'
    AGGREGATION_WINDOW='1 day';

In [ ]:
#Click the generated link to view your model in the model regsitry and check out the model monitors!
st.write(f'https://app.snowflake.com/{org_name}/{account_name}/#/data/databases/{DB}/schemas/{SCHEMA}/model/{model_name.upper()}')

In [ ]:
SELECT * FROM TABLE(MODEL_MONITOR_DRIFT_METRIC(
'MORTGAGE_LENDING_BASE_MODEL_MONITOR', -- model monitor to use
'DIFFERENCE_OF_MEANS', -- metric for computing drift
'XGB_BASE_PREDICTION', -- comlumn to compute drift on
'1 DAY',  -- day granularity for drift computation
DATEADD(DAY, -90, CURRENT_DATE()), -- end date
DATEADD(DAY, -60, CURRENT_DATE()) -- start date
)
)

# SPCS Deployment setup (OPTIONAL)
## This is disabled by default but uncommenting the below code cells will allow a user to 

- ### Create a new compute pool with 3 XL CPU nodes
- ### Deploys a service on top of our existing HPO model version
- ### Tests out inference on newly created container service


In [ ]:
cp_name = "MORTGAGE_LENDING_INFERENCE_CP"
num_spcs_nodes = '2'
spcs_instance_family = 'CPU_X64_L'
service_name = 'MORTGAGE_LENDING_PREDICTION_SERVICE'

current_database = session.get_current_database().replace('"', '')
current_schema = session.get_current_schema().replace('"', '')
extended_service_name = f'{current_database}.{current_schema}.{service_name}'

In [ ]:
session.sql(f"alter compute pool if exists {cp_name} stop all").collect()
session.sql(f"drop compute pool if exists {cp_name}").collect()
session.sql(f"create compute pool {cp_name} min_nodes={num_spcs_nodes} max_nodes={num_spcs_nodes} instance_family={spcs_instance_family} auto_resume=True auto_suspend_secs=300").collect()
session.sql(f"describe compute pool {cp_name}").show()

In [ ]:
#note this may take up to 5 minutes to run

mv_opt.create_service(
    service_name=extended_service_name,
    service_compute_pool=cp_name,
    ingress_enabled=True,
    max_instances=int(num_spcs_nodes)
)

In [ ]:
model_registry.get_model(f"MORTGAGE_LENDING_MLOPS_{VERSION_NUM}").show_versions()

In [ ]:
mv_container = model_registry.get_model(f"MORTGAGE_LENDING_MLOPS_{VERSION_NUM}").default
mv_container.run(test, function_name = "predict", service_name = "MORTGAGE_LENDING_PREDICTION_SERVICE").rename('"output_feature_0"', 'XGB_PREDICTION')

In [ ]:
SHOW ENDPOINTS IN SERVICE E2E_SNOW_MLOPS_DB.MLOPS_SCHEMA.MORTGAGE_LENDING_PREDICTION_SERVICE

In [ ]:
#Stop the service to save costs
# session.sql(f"alter compute pool if exists {cp_name} stop all").collect()

## Conclusion 

#### 🛠️ Snowflake Feature Store tracks feature definitions and maintains lineage of sources and destinations 🛠️
#### 🚀 Snowflake Model Registry gives users a secure and flexible framework to log models, tag candidates for production, and run inference and explainability jobs 🚀
#### 📈 ML observability in Snowflake allows users to montior model performance over time and detect model, feature, and concept drift 📈
#### 🔮 All models logged in the Model Registry can be accessed for inference, explainability, lineage tracking, visibility and more 🔮
